In [1]:
import os
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import L1L2

In [2]:
MODEL = "encoder-decoder"

config = {
    "N_PREV": 90,
    "N_FWD": 30,
    
    "ENCODER_SIZE": 16,
    "DECODER_SIZE": 16,
    
    "LR": 0.001,
    "BATCH_SIZE": 64,
    "EPOCHS": 30,
}

In [3]:
import wandb
from wandb.keras import WandbCallback
from secrets import WANDB
wandb.login(key=WANDB)

wandb: Currently logged in as: all-off-nothing (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\kiern/.netrc


True

## **Load Data Function**

In [4]:
def get_data(fold):
    with open(f'./data/fold{fold}_data.npy', mode='rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_val = np.load(f)
        y_val = np.load(f)
    with open(f'./data/fold{fold}_normalizer.pkl', mode='rb') as f:
        normalizer = pickle.load(f)
    return X_train, y_train, X_val, y_val, normalizer
        

## **Model Functions**

In [5]:
def simple_encoder_decoder():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1))
    x, hidden_state, cell_state = tf.keras.layers.LSTM(config["ENCODER_SIZE"], 
                                                       return_sequences=True, 
                                                       return_state=True, 
                                                       recurrent_initializer='glorot_uniform', 
                                                       activity_regularizer=L1L2(l1=0.000001, l2=0.000001))(inputs)
    
    decoder = tf.keras.layers.LSTM(config["DECODER_SIZE"],
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform', 
                                   activity_regularizer=L1L2(l1=0.000001, l2=0.000001))
    dropout = tf.keras.layers.Dropout(0.2)
    decoder_output = tf.keras.layers.Dense(1)
    all_outputs = []
    last_value = tf.expand_dims(inputs[:, -1, 0:1], 1)
    states = [hidden_state, cell_state]
    for i in range(config["N_FWD"]):
        x, hidden_state, cell_state = decoder(last_value, initial_state=states)
        states=[hidden_state, cell_state]
        x = dropout(x)
        x = decoder_output(x)
        last_value = x
        all_outputs.append(x)
    outputs = tf.keras.layers.Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mse", metrics=["mae"], optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"]))
    return model

## **Train Loop**

In [6]:
def train_model(model, X_train, y_train, X_val, y_val):
    reducer = tf.keras.callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.1, patience=2, mode='min', cooldown=1)
    stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, mode='min', restore_best_weights=True)
    model.fit(X_train, y_train,
              batch_size=config["BATCH_SIZE"],
              epochs=config["EPOCHS"], 
              callbacks=[reducer, stopper],
              validation_data=(X_val, y_val),
              validation_batch_size=config["BATCH_SIZE"],
             shuffle=True)
    return model

In [7]:
def train():
    for fold in range(1, 4):
        run = wandb.init(project="time-series-methods", entity="kmcguigan", group=f"{MODEL}-model", config=config, job_type="train")
        run.name = f'{MODEL}-fold-{fold}'
        X_train, y_train, X_val, y_val, normalizer = get_data(fold)
        model = simple_encoder_decoder()
        model = train_model(model, X_train, y_train, X_val, y_val)
        run.finish()
    return

In [ ]:
train()

Epoch 1/30
2484/2484 [==============================] - 144s 41ms/step - loss: 0.4057 - mae: 0.4917 - val_loss: 0.2958 - val_mae: 0.4215 - lr: 0.0010
Epoch 2/30
2484/2484 [==============================] - 91s 37ms/step - loss: 0.2575 - mae: 0.3877 - val_loss: 0.1899 - val_mae: 0.3296 - lr: 0.0010
Epoch 3/30
2484/2484 [==============================] - 87s 35ms/step - loss: 0.2261 - mae: 0.3607 - val_loss: 0.2355 - val_mae: 0.3667 - lr: 0.0010
Epoch 4/30
2484/2484 [==============================] - 85s 34ms/step - loss: 0.2003 - mae: 0.3376 - val_loss: 0.1482 - val_mae: 0.2857 - lr: 0.0010
Epoch 5/30
2484/2484 [==============================] - 86s 35ms/step - loss: 0.1799 - mae: 0.3181 - val_loss: 0.1496 - val_mae: 0.2864 - lr: 0.0010
Epoch 6/30
2484/2484 [==============================] - 87s 35ms/step - loss: 0.1701 - mae: 0.3089 - val_loss: 0.1315 - val_mae: 0.2693 - lr: 0.0010
Epoch 7/30
2484/2484 [==============================] - 87s 35ms/step - loss: 0.1819 - mae: 0.3202 - val_

Epoch 1/30
3162/3162 [==============================] - 216s 52ms/step - loss: 0.3220 - mae: 0.4301 - val_loss: 0.1739 - val_mae: 0.3076 - lr: 0.0010
Epoch 2/30
3162/3162 [==============================] - 150s 47ms/step - loss: 0.1896 - mae: 0.3278 - val_loss: 0.1474 - val_mae: 0.2828 - lr: 0.0010
Epoch 3/30
3162/3162 [==============================] - 151s 48ms/step - loss: 0.1759 - mae: 0.3148 - val_loss: 0.1590 - val_mae: 0.2902 - lr: 0.0010
Epoch 4/30
3162/3162 [==============================] - 161s 51ms/step - loss: 0.1779 - mae: 0.3168 - val_loss: 0.1444 - val_mae: 0.2746 - lr: 0.0010
Epoch 5/30
3162/3162 [==============================] - 161s 51ms/step - loss: 0.1879 - mae: 0.3242 - val_loss: 0.1573 - val_mae: 0.2918 - lr: 0.0010
Epoch 6/30
3162/3162 [==============================] - 151s 48ms/step - loss: 0.1611 - mae: 0.3007 - val_loss: 0.1467 - val_mae: 0.2789 - lr: 0.0010
Epoch 7/30
3162/3162 [==============================] - 169s 53ms/step - loss: 0.1498 - mae: 0.2891 